
<div class="alert alert-block alert-info">
<h1>Data set-up and preprocessing</h1>

In [1]:
import warnings                  

import pandas as pd

from helper import parameter_mixer
from cross_validation import BaseTimeSeriesModelCrossValidation
             
warnings.filterwarnings('ignore')

# [TODO] you need to change this path to your local data path
data_path = './data/train.csv'

df = pd.read_csv('./data/train.csv')
df['Datetime'] = pd.to_datetime(df['Datetime']).dt.tz_localize(None)
df = df.sort_values(by='Datetime').rename(columns={'Datetime' : 'ds', 'ActivePower' : 'y'})

# [TODO] We would use different method to aggregate features (X)
df = df.set_index('ds').resample('D').mean()

last_date = df.index[-1]

In [ ]:
cv = BaseTimeSeriesModelCrossValidation(num_fold=5, rolling_size=10, validation_size=15)

<div class="alert alert-block alert-warning">  
<h3> [Demo] SARIMAX model (Inseong Han)</h3>

In [ ]:
from sarimax_model import SARIMAX

model = SARIMAX(m=7)
p_values = [0, 1]
d_values = [0]
q_values = [0, 1]
P_values = [1, 2]
D_values = [1]
Q_values = [1, 2]
t_values = ['n']


params = parameter_mixer(
    ['p', 'd', 'q', 'P', 'D', 'Q', 't'],
    [p_values, d_values, q_values, P_values, D_values, Q_values, t_values]
)

training_history, best_p, best_rmse = cv.cross_validation(df['y'], model, params)

# Train the 
model.reset()
model.fit(df['y'], best_p)

<div class="alert alert-block alert-warning">  
<h3> Prophet model (Inseong Han)</h3>

In [ ]:
from prophet_model import ProphetModel

model = ProphetModel()
params = [None]

training_history, best_p, best_rmse = cv.cross_validation(df['y'], model, params)

model.reset()
model.fit(df['y'], best_p)

<div class="alert alert-block alert-warning">  
<h3> [Submission]</h3>

In [ ]:
from datetime import timedelta
submission = model.forecast(15).reset_index(drop=True)
submission.index = last_date + pd.to_timedelta(submission.index + 1, unit='D')
submission.index.name = 'Date'
submission.name = 'Forecasting'
submission.to_csv('submission.csv', index=True)